In [1]:
import scvelo as scv
import loompy
import igraph as ig
import anndata as ad
import pandas as pd
import re
import scanpy as sc 
import matplotlib.pyplot as plt 
import numpy as np 
import scipy as sp
import seaborn as sb
import harmonypy
from numpy import inf

adata = sc.read_h5ad(filename = "/omics/groups/OE0540/internal/users/ruehle/rnavelocity/velocyto/scvelo_output/use/2000DEpercelltype_velocities_neighbors_pca.h5ad")

#celltype to aggregate
celltype = "defendoderm"

#define aggregation function 
def grouped_obs_mean(adata, group_key, layer=None, gene_symbols=None):
    if layer is not None:
        getX = lambda x: x.layers[layer]
    else:
        getX = lambda x: x.X
    if gene_symbols is not None:
        new_idx = adata.var[idx]
    else:
        new_idx = adata.var_names

    grouped = adata.obs.groupby(group_key)
    out = pd.DataFrame(
        np.zeros((adata.shape[1], len(grouped)), dtype=np.float64),
        columns=list(grouped.groups.keys()),
        #index=adata.var_names
        index = adata.var["Accession"]
    )

    for group, idx in grouped.indices.items():
        X = getX(adata[idx])
        out[group] = np.ravel(X.mean(axis=0, dtype=np.float64))
    return out

In [40]:
#write phenotype file 
blub = pd.DataFrame()
cells = pd.DataFrame()
sample_ids = []

cells = adata[adata.obs['celltype'] == celltype, :]
#blub = grouped_obs_mean(cells, ["donor", "run"], layer = "matrix")
blub = grouped_obs_mean(cells, ['donor', 'run'], layer = 'spliced')
blub = blub.T

#fix row/colnames of aggregates 
for name in blub.index:
   item = name[0] + "-" + name[1]
   sample_ids.append(item)

neue_sample_ids = []
for name in sample_ids:
    ohne_run = name.replace("run","")
    neue_sample_ids.append(ohne_run)

#print(neue_sample_ids[0:10])

blub.insert(0, 'feature_id', neue_sample_ids)
spliced = blub.T
out_path = "/omics/groups/OE0540/internal/users/ruehle/rnavelocity/vqtls/data/velocyto/logodds_velocity/celltypes/" + celltype + "/input/"

spliced = spliced.drop('feature_id')
spliced = spliced.to_numpy(dtype = float)
#spliced = blub.dropna()
spliced.shape

blub.to_csv(out_path + "pheno_file_logodds_velocity_" + celltype + '_DE_genes.txt', header=False, index=True, sep='\t', mode='a')

View of AnnData object with n_obs × n_vars = 8644 × 7459
    obs: 'Unnamed: 0', 'run', 'batch', 'cell', 'timepoint', 'donor', 'exp_batch', 'annas_pseudotime', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_counts_highlyvariable', 'log_counts_highlyvariable', 'n_genes_highlyvariable', 'louvain_1.0', 'louvain_0.6', 'louvain_0.4', 'louvain_1.4', 'n_counts', 'velocity_self_transition', 'root_cells', 'end_points', 'velocity_pseudotime', 'latent_time', 'pc_pseudotime', 'pc_x', 'pc_y', 'pc_data_x', 'pc_data_y', 'poor_differentiator', 'celltype'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_', 'fit_scaling', 'fit_std_u', 'fit_std_s', 'fit_likelihood', 'fit_u0', 'fit_s0', 'fit_pval_steady', 'fit_steady_u', 'fit_steady_s', 'fit_variance', 'fit_alignment_scaling', 'fit_r2', 'velocity_genes'
    uns: 'neighbors', 'pca', 'rank_genes_groups', 're

(7459, 264)

In [41]:
#write phenotype file 
blub = pd.DataFrame()
cells = pd.DataFrame()
sample_ids = []

cells = adata[adata.obs['celltype'] == celltype, :]

#blub = grouped_obs_mean(cells, ["donor", "run"], layer = "matrix")
#blub = grouped_obs_mean(cells, ["donor", "exp_batch"])
blub = grouped_obs_mean(cells, ['donor', 'run'], layer = 'unspliced')
gene_ids = blub.index
blub = blub.T


for name in blub.index:
   item = name[0] + "-" + name[1]
   sample_ids.append(item)

neue_sample_ids = []
for name in sample_ids:
    ohne_run = name.replace("run","")
    neue_sample_ids.append(ohne_run)

print(neue_sample_ids[0:10])

blub.insert(0, 'feature_id', neue_sample_ids)
unspliced = blub.T
out_path = "/omics/groups/OE0540/internal/users/ruehle/rnavelocity/vqtls/data/velocyto/logodds_velocity/celltypes/" + celltype + "/input/"
unspliced = unspliced.drop('feature_id')
unspliced = unspliced.to_numpy(dtype = float)

#unspliced = blub.dropna()
unspliced.shape
#blub[blub == -inf] = 0
#blub[blub == inf] = 0

View of AnnData object with n_obs × n_vars = 8644 × 7459
    obs: 'Unnamed: 0', 'run', 'batch', 'cell', 'timepoint', 'donor', 'exp_batch', 'annas_pseudotime', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_counts_highlyvariable', 'log_counts_highlyvariable', 'n_genes_highlyvariable', 'louvain_1.0', 'louvain_0.6', 'louvain_0.4', 'louvain_1.4', 'n_counts', 'velocity_self_transition', 'root_cells', 'end_points', 'velocity_pseudotime', 'latent_time', 'pc_pseudotime', 'pc_x', 'pc_y', 'pc_data_x', 'pc_data_y', 'poor_differentiator', 'celltype'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_', 'fit_scaling', 'fit_std_u', 'fit_std_s', 'fit_likelihood', 'fit_u0', 'fit_s0', 'fit_pval_steady', 'fit_steady_u', 'fit_steady_s', 'fit_variance', 'fit_alignment_scaling', 'fit_r2', 'velocity_genes'
    uns: 'neighbors', 'pca', 'rank_genes_groups', 're

(7459, 264)

In [42]:
#merge selected cells 

crude_velo = unspliced/(spliced + unspliced)
logit_velocity = np.log(crude_velo/(1-crude_velo))

logit_velocity[logit_velocity == -inf] = "NaN"
logit_velocity[logit_velocity == inf] = "NaN"

#write phenotype file 
logit_velocity_panda = pd.DataFrame()
logit_velocity_panda = pd.DataFrame(data=logit_velocity)

logit_velocity_panda.head()


neue_sample_ids = pd.Series(neue_sample_ids)
logit_velocity_panda.loc[-1] = neue_sample_ids
# adding a row
logit_velocity_panda.index = logit_velocity_panda.index + 1 # shifting index
logit_velocity_panda

logit_velocity_panda = logit_velocity_panda.sort_index()
logit_velocity_panda.index = gene_ids.insert(0,"feature_id")

logit_velocity_panda

out_path = "/omics/groups/OE0540/internal/users/ruehle/rnavelocity/vqtls/data/velocyto/logodds_velocity/celltypes/" + celltype + "/input/"
logit_velocity_panda.to_csv(out_path + "pheno_file_logodds_velocity_" + celltype + '_DE_genes.txt', header=False, index=True, sep='\t', mode='a')



In [43]:
#sample mapping file
#maps genotype names to sample names 

donormeta = pd.read_csv('/icgc/dkfzlsdf/analysis/B260/projects/HipSci/openAccess/endoderm_differentation/metadata/HipSci_donor_annotation.tsv', sep = ';', header = 0)

subdf = pd.DataFrame()
final = pd.DataFrame()
subdf['sample_id'] = logit_velocity_panda.loc["feature_id"]

subdf['sample_map'] = "zero"
subdf['donor'] = "donor"

#remove run donor thing and put it into column donor of subdf 
for index, row in subdf.iterrows():
    donor_o = str(row['sample_id'])
    donor_only = donor_o[:-6]
    row['donor'] = donor_only

for sample_index, sample_row in subdf.iterrows():
    sample = sample_row['sample_id']
    for genomic_index, genomic_row in donormeta.iterrows():
        if genomic_row['DonorLine'] in sample:
            sample_row['sample_map'] = genomic_row["Sample2"]
        else:
            None

final = subdf[['sample_map', 'sample_id']]
final.shape

final.to_csv(out_path + "sample_mapping_file_" + celltype + '.txt', header=False, index=False, sep='\t', mode='a')